In [1]:
import matplotlib
import matplotlib.pylab as plt
import glob
import numpy
import math
from joblib import Parallel, delayed

# Configuration
RUN_ID = "1"
RUNNING_TIME = 20

In [2]:
def res_dir(rid):
    return "results/" + rid

In [3]:
def process_log(filename):
    print("Processing", filename)

    publications = {}
    latencies = {}
    max_time = 0
    sync_pack = 0
    sync_byte = 0
    
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('SYNC_'):
                val = int(line.split('=')[1])
                if line.startswith('SYNC_PACK'):
                    sync_pack = val
                if line.startswith('SYNC_BYTE'):
                    sync_byte = val
                    
                continue
            
            line = [x.strip() for x in line.split(',')]
            
            time = float(line[0])
            node = line[1]
            mtype = line[2]
            mid = line[3]
            
            if mtype == 'PUB':
                publications[mid] = time
            
            if mtype == 'RECV':
                if mid not in latencies:
                    latencies[mid] = []
                latencies[mid].append(time - publications[mid])
                
            max_time = max(max_time, time)
    
    return latencies, max_time, sync_pack, sync_byte

In [4]:
method = 'FULL'
grid = '4'
loss = '0'
logfiles = glob.glob("exp_log_files_josh/" +
                         method + "_Method-" +
                         grid + "_Row-" +
                         grid + "_Col-"+
                         "1000_MS_INTER_SLOW-100_MS_INTER_FAST-"+
                         "*_Nodes_Pub_Fast-"+
                         "*_RECENT_PUB-" +
                         "*_RANDOM_PUB-10_S_STOP-" +
                         loss + "_DROP_RATE*.log")
print(logfiles)
process_log(logfiles[0])

['exp_log_files_josh\\FULL_Method-4_Row-4_Col-1000_MS_INTER_SLOW-100_MS_INTER_FAST-0_Nodes_Pub_Fast-0_RECENT_PUB-99999_RANDOM_PUB-10_S_STOP-0_DROP_RATE-4_MTU.log']
Processing exp_log_files_josh\FULL_Method-4_Row-4_Col-1000_MS_INTER_SLOW-100_MS_INTER_FAST-0_Nodes_Pub_Fast-0_RECENT_PUB-99999_RANDOM_PUB-10_S_STOP-0_DROP_RATE-4_MTU.log


({'/participant2::1': [110.048, 120.528, 130.576, 130.576, 3187.7],
  '/participant3::1': [110.048, 110.48, 120.528, 120.528, 3187.08],
  '/participant4::1': [110.048, 110.048, 110.48, 120.528, 3187.08],
  '/participant5::1': [110.048, 120.096, 120.528, 130.576, 3187.08],
  '/participant6::1': [110.048, 120.096, 121.008, 131.056, 3187.08],
  '/participant4::2': [11.097999999999956,
   11.097999999999956,
   11.984000000000037,
   22.081999999999994,
   2385.08],
  '/participant3::2': [11.097999999999956,
   11.984000000000037,
   22.081999999999994,
   22.081999999999994,
   2341.08],
  '/participant2::2': [11.145999999999958,
   22.562000000000012,
   32.610000000000014,
   32.610000000000014,
   2318.7],
  '/participant6::2': [11.097999999999956,
   21.197000000000003,
   22.081999999999994,
   32.18100000000004,
   2262.08],
  '/participant5::2': [11.097999999999956,
   21.197000000000003,
   22.081999999999994,
   32.18100000000004,
   2232.08],
  '/participant3::3': [11.0999999999

In [11]:
jobs = []
def add_plot(r, label):
    lists = sorted(r.items())
    x, y = zip(*lists)
    plt.plot(x, y, label=label)
def run_job(rid, label, scale):
    logfiles = glob.glob(res_dir(rid) + "/*.log")
    percentile = 90

    results50 = {}
    results90 = {}
    results_pack = {}
    results_byte = {}

    def set_p(res, max_time, sync_pack, sync_byte):
        nums50 = []
        nums90 = []
        for r in res:
            nums50.append(numpy.percentile(res[r], 50))
            nums90.append(numpy.percentile(res[r], 90))

        k = len(res)/(max_time/1000)
        results50[k] = numpy.average(nums50) / scale
        results90[k] = numpy.average(nums90) / scale
        results_pack[k] = sync_pack
        results_byte[k] = sync_byte

    for lf in logfiles:
        set_p(*process_log(lf))

    return results90, results50, label, results_pack, results_byte
def add_run(rid, label, scale):
    jobs.append(delayed(run_job)(rid, label, scale))
def run_jobs():
    res = Parallel(n_jobs=-1)(jobs)
    for r in res:
        add_plot(r[0], r[2])

[(<function run_job at 0x000001B3CCEE4310>, ('4', '4 nodes', 2.0), {}), (<function run_job at 0x000001B3CCEE4310>, ('4', '4 nodes', 1), {})]
[({}, {}, '4 nodes', {}, {}), ({}, {}, '4 nodes', {}, {})]


In [14]:
add_run("4", "4 nodes", 1)
run_jobs()
    
plt.xlabel("Total publications per second")
plt.ylabel("Latency")
plt.legend()
plt.show()

ValueError: not enough values to unpack (expected 2, got 0)

In [5]:
_='''
def add_plot(r, label):
    lists = sorted(r.items())
    x, y = zip(*lists)
    plt.plot(x, y, label=label)

jobs = []
    
def run_job(rid, label, scale):
    logfiles = glob.glob(res_dir(rid) + "/*.log")
    percentile = 90

    results50 = {}
    results90 = {}
    results_pack = {}
    results_byte = {}

    def set_p(res, max_time, sync_pack, sync_byte):
        nums50 = []
        nums90 = []
        for r in res:
            nums50.append(numpy.percentile(res[r], 50))
            nums90.append(numpy.percentile(res[r], 90))

        k = len(res)/(max_time/1000)
        results50[k] = numpy.average(nums50) / scale
        results90[k] = numpy.average(nums90) / scale
        results_pack[k] = sync_pack
        results_byte[k] = sync_byte

    for lf in logfiles:
        set_p(*process_log(lf))

    return results90, results50, label, results_pack, results_byte
    
def add_run(rid, label, scale):
    jobs.append(delayed(run_job)(rid, label, scale))

def run_jobs():
    res = Parallel(n_jobs=-1)(jobs)
    for r in res:
        add_plot(r[0], r[2])
    
# add_run("4", "4 nodes", math.sqrt(4))
add_run("8-n", "8 nodes", math.sqrt(8))
#add_run("16-n", "16 nodes", math.sqrt(16))
#add_run("32-n", "32 nodes", math.sqrt(32))
# add_run("64-n", "64 nodes", math.sqrt(64))

#add_run("8", "8 nodes (frag)", math.sqrt(8))
#add_run("16", "16 nodes (frag)", math.sqrt(16))
#add_run("32", "32 nodes (frag)", math.sqrt(32))
# add_run("64", "64 nodes (frag)", math.sqrt(64))
# add_run("128", "128 nodes", math.sqrt(128))

run_jobs()
    
plt.xlabel("Total publications per second")
plt.ylabel("Latency")
plt.legend()
plt.show()
'''

In [7]:
fig = matplotlib.pyplot.gcf()

def add_plot(r, label, marker):
    lists = sorted(r.items())
    x, y = zip(*lists)
    plt.plot(x, y, label=label, marker=marker)

jobs = []
    
def run_job(grid, method, loss, label, scale, marker):
    logfiles = glob.glob("exp_log_files/" +
                         method + "_Method-" +
                         grid + "_Row-" +
                         grid + "_Col-"+
                         "1000_MS_INTER_SLOW-100_MS_INTER_FAST-"+
                         "*_Nodes_Pub_Fast-"+
                         "*_RECENT_PUB-" +
                         "*_RANDOM_PUB-10_S_STOP-" +
                         loss + "_DROP_RATE*.log")
    print(logfiles)

    percentile = 90

    results50 = {}
    results90 = {}
    results_pack = {}
    results_byte = {}

    def set_p(res, max_time, sync_pack, sync_byte):
        nums50 = []
        nums90 = []
        for r in res:
            nums50.append(numpy.percentile(res[r], 50))
            nums90.append(numpy.percentile(res[r], 90))

        print(max_time)
        k = len(res)/(max_time/1000)
        results50[k] = numpy.average(nums50) / scale
        results90[k] = numpy.average(nums90) / scale
        results_pack[k] = sync_pack
        results_byte[k] = sync_byte

    for lf in logfiles:
        print(lf)
        set_p(*process_log(lf))

    return results90, results50, label, results_pack, results_byte, marker
    
def add_run(*args):
    jobs.append(delayed(run_job)(*args))

#GRAPH_TYPE = "latency"
#GRAPH_TYPE = "packets"
GRAPH_TYPE = "bytes"
LOSS = "0.5"
GRID = "4"
    
def run_jobs():
    res = Parallel(n_jobs=-1)(jobs)
    for r in res:
        # 0 for 90%tile
        # 1 for 50%tile
        # 3 for #packets
        # 4 for bytes
        index = 0
        if GRAPH_TYPE == "packets":
            index = 3
        elif GRAPH_TYPE == "bytes":
            index = 4

        add_plot(r[index], r[2], r[5])

# add_run(GRID, "BASE", LOSS, "{0}x{0}, Base".format(GRID), math.sqrt(1), 'o')
add_run(GRID, "FULL", LOSS, "{0}x{0}, FragFull".format(GRID), math.sqrt(1), 'v')
# add_run(GRID, "RAND", LOSS, "{0}x{0}, Rand".format(GRID), math.sqrt(1), '^')
# add_run(GRID, "RAND_RECENT", LOSS, "{0}x{0}, RandRecent".format(GRID), math.sqrt(1), 's')

run_jobs()
    
plt.xlabel("Total publications per second")

if GRAPH_TYPE == "latency":
    plt.ylabel("Latency")
elif GRAPH_TYPE == "packets":
    plt.ylabel("Overhead (packets)")
else:
    plt.ylabel("Overhead (bytes)")

fig.set_size_inches(6, 2.5)
plt.grid()
plt.legend()
fig.show()
fig.savefig('fig.pdf',bbox_inches='tight', pad_inches=0)  

ValueError: not enough values to unpack (expected 2, got 0)

<Figure size 432x288 with 0 Axes>